# Tabulation of Fractional Cover data within tiled shapefile polygons

**What does this notebook do?**

This notebook is a pilot collaboration between Geoscience Australia and Australian Bureau of Statistics. The purpose of the notebook is to use a shapefile polygon boundaries to load fractional cover dataset, complete zonal statistics and tabulate the results.

**Requirements**

You need to run the following commands from the command line prior to launching jupyter notebooks from the same terminal so that the required libraries and paths are set:

`module use /g/data/v10/public/modules/modulefiles`

`module load dea/20181213`


**Background**

Data from the Landsat 5,7 and 8 satellite missions are accessible through Digital Earth Australia (DEA). The code snippets in this notebook will let you retrieve and plot the Fractional Cover (FC25) data stored in DEA.


**How to use this notebook**

A basic understanding of any programming language is desirable but one doesn't have to be an expert Python programmer to manipulate the code to get and display the data.This doc applies to the following Landsat satellites, Fractional Cover bands and the WOfS dataset:

- Landsat 5
- Landsat 7
- Landsat 8
- PV - Photosythetic vegetation
- NPV - Non-Photosythetic vegetation
- BS - Bare Soil
- UE - Unmixing Error
- Water Observations from Space (WOFs)
- WOfS Feature Layer (WOFL)

**Bugs still to fix**

- Qaulity metrics for tiled/large SA2
- Check area of SA2 for Percentage calculation


**Errors or bugs**

If you find an error or bug in this notebook, please contact erin.telfer@ga.gov.au.


## 1. Import Libraries

In [2]:
%matplotlib inline

from datetime import time, datetime
import os.path

from matplotlib import pyplot as plt
import pandas as pd
import numpy
import csv
import xarray as xr
import rasterio
import rasterio.features
import fiona
from datetime import datetime
import dask
from dask.delayed import delayed
from dask.distributed import LocalCluster, Client
import tempfile

import datacube
from datacube import Datacube
from datacube.virtual import construct, construct_from_yaml
from datacube.ui.task_app import year_splitter
from datacube.utils.geometry import CRS

print(f"Cell finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Cell finished at 2019-05-17 12:59:10


## 2. Define directories and years of interest

In [3]:
#Set folder  and SA2 shapefile locations
# shapefile_path = '/g/data/r78/ext547/abs/input/SA_2016_clipped_albers_3577.shp'
shapefile_path = '/g/data/r78/ext547/abs/input/SA_2016 _clipped_113031266_3577.shp'

output_path = '/g/data/r78/ext547/abs/output/'
name_of_output_file ='tabulate_FC_large_SA2_tiled_test'

In [4]:
#Specify time years of interest
start_year, end_year = 2005, 2006
time_range = (str(start_year), str(end_year))
print(time_range)

('2005', '2006')


## 3. Set up a local dask cluster
Some calculations take more memory than is available on a system.  By breaking the data up into chunks, we can chain a sequence of operations together, and work on the data a small piece at a time.

This lets several processes work at the same time, and manage total memory usage for the calculations.

In more advanced setups, we can distribute the work across multiple computers, using all of their memory and CPU power.

* We set `n_workers` to be the number of worker applications we want to run in the background, doing the processing of the chunk-based steps we have chained together.
* The `mem_per_worker` setting defines how much memory at most is available to each of the workers.
* `chunk_size` sets the width and height of the chunk in pixels of the size will break up the data into.

VDI has 8 CPUs available, and a total of 32GB of memory, but you will typically be sharing those with several (2-10) other users.   

In [4]:
#Set up dask cluster
n_workers = 7
threads_per_worker=1
mem_per_worker = 8e9  # 8e9 is 8GB (8,000,000,000 bytes)

chunk_size = {'time': 1, 'x': 2000, 'y': 2000}

In [5]:
cluster = LocalCluster(local_dir=tempfile.gettempdir(), 
                       n_workers=n_workers, 
                       threads_per_worker=threads_per_worker,
                       memory_limit=mem_per_worker)
client = Client(cluster)
dask.config.set(get=client.get)
client

Client Scheduler: tcp://127.0.0.1:42178 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 7 Cores: 7 Memory: 56.00 GB


We also get a dashboard to see how the system is running, by clicking the link above after the cell has been run.

## 4. Connect to the Datacube 

In [ ]:
dc = Datacube()
print(f"Cell finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 5. Construct the virtual product

In [ ]:
#Remove Landsat 7 scenes with the Scan Line Correction (SLC) missing data
LS7_BROKEN_DATE = datetime(2003, 5, 31)
is_pre_slc_failure = lambda dataset: dataset.center_time < LS7_BROKEN_DATE
print(f"Cell finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
#Create function to ensure wofls in correct format
def wofls_fuser(dest, src):
    where_nodata = (src & 1) == 0
    numpy.copyto(dest, src, where=where_nodata)
    return dest
print(f"Cell finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
#Create virtual product so that datacube data can be loaded effectively within memory
fc_and_water_yaml = """
        juxtapose:
          - collate:
              - transform: apply_mask
                mask_measurement_name: pixelquality
                preserve_dtype: false
                input:
                    juxtapose:
                      - product: ls5_fc_albers
                        group_by: solar_day
                        measurements: [PV, NPV, BS]
                      - transform: make_mask
                        input:
                            product: ls5_pq_albers
                            group_by: solar_day
                            fuse_func: datacube.helpers.ga_pq_fuser
                        flags:
                            ga_good_pixel: true
                        mask_measurement_name: pixelquality
              - transform: apply_mask
                mask_measurement_name: pixelquality
                preserve_dtype: false
                input:
                    juxtapose:
                      - product: ls7_fc_albers
                        group_by: solar_day
                        measurements: [PV, NPV, BS]
                        # dataset_predicate: __main__.is_pre_slc_failure
                      - transform: make_mask
                        input:
                            product: ls7_pq_albers
                            group_by: solar_day
                            fuse_func: datacube.helpers.ga_pq_fuser
                        flags:
                            ga_good_pixel: true
                        mask_measurement_name: pixelquality
              - transform: apply_mask
                mask_measurement_name: pixelquality
                preserve_dtype: false
                input:
                    juxtapose:
                      - product: ls8_fc_albers
                        group_by: solar_day
                        measurements: [PV, NPV, BS]
                      - transform: make_mask
                        input:
                            product: ls8_pq_albers
                            group_by: solar_day
                            fuse_func: datacube.helpers.ga_pq_fuser
                        flags:
                            ga_good_pixel: true
                        mask_measurement_name: pixelquality
          - transform: make_mask
            input:
                product: wofs_albers
                group_by: solar_day
                fuse_func: __main__.wofls_fuser
            flags:
                wet: true
            mask_measurement_name: water
"""
fc_and_water = construct_from_yaml(fc_and_water_yaml)

## 6. Set up functions

In [ ]:
def geometry_mask(geoms, geobox, all_touched=False, invert=False, chunks=None):
    """
    Create a mask from shapes.

    By default, mask is intended for use as a
    numpy mask, where pixels that overlap shapes are False.
    :param list[Geometry] geoms: geometries to be rasterized
    :param datacube.utils.GeoBox geobox:
    :param bool all_touched: If True, all pixels touched by geometries will be burned in. If
                             false, only pixels whose center is within the polygon or that
                             are selected by Bresenham's line algorithm will be burned in.
    :param bool invert: If True, mask will be True for pixels that overlap shapes.
    """
    data = rasterio.features.geometry_mask([geom.to_crs(geobox.crs) for geom in geoms],
                                           out_shape=geobox.shape,
                                           transform=geobox.affine,
                                           all_touched=all_touched,
                                           invert=invert)
    if chunks is not None:
        data = dask.array.from_array(data, chunks=tuple(chunks[d] for d in geobox.dims))
        
    coords = [xr.DataArray(data=coord.values, name=dim, dims=[dim], attrs={'units': coord.units}) 
              for dim, coord in geobox.coords.items()]
    return xr.DataArray(data, coords=coords)
print(f"Cell finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
def get_shapes(shape_file):
    """
    Extract spatial inforamtion from polygons within shapefile
    """
    with fiona.open(shape_file) as shapes:
        crs = datacube.utils.geometry.CRS(shapes.crs_wkt)
        for shape in shapes:
            if shape['geometry'] is None:
                continue
            geom = datacube.utils.geometry.Geometry(shape['geometry'], crs=crs)
            geom = geom.to_crs(CRS('EPSG:3577'))
            yield geom, shape['properties']
print(f"Cell finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
def fc_and_water_summary(data, mask_int):
    """
    Calculate the percentage and the area of each FC component, 
    water, and unclassified data within each SA2 boundary. 
    """
    # Where there is water, set the FC bands to 0%
    valid_data = numpy.isfinite(data['BS'])
    fc = data[['PV', 'NPV', 'BS']].where(data.water==0, other=0)
    fc['water'] = data.water.where(valid_data) * numpy.float32(100)
    fc = fc.apply(lambda data_array: data_array.clip(0, 100).where(valid_data))

    has_data = valid_data.groupby(valid_data.time.astype('datetime64[M]'), squeeze=False).sum(dim='time', skipna=False)
    has_data = has_data.sum(dim=['x','y'], skipna=True) / (mask_int / 100)
    
    # Flatten to a monthly mean
    fc = fc.groupby(fc.time.astype('datetime64[M]'), squeeze=False).mean(dim='time', skipna=True)
    # Calculate percentage of cover based on area of mask
    percentage = fc.sum(dim=['x','y']) * (100 / mask_int)
    for da in percentage.data_vars.values():
        da.attrs['units'] = '%'

#     fc_sum = fc.sum(dim=['x','y'])
#     fc_sum = fc_sum.PV + fc_sum.NPV + fc_sum.BS + fc_sum.water
    
#     unclass_size = mask_int - fc_sum
#     unclass_percentage = unclass_size * (100 / mask_int)
#     fc['unclassified'] = unclass_percentage 
#     percentage['unclassified'] = unclass_percentage
    fc['mask_int'] = mask_int
    
#     print(f"   PV = {int(percentage.PV.values)}%, NPV = {int(percentage.NPV.values)}%, BS = {int(percentage.BS.values)}%, water = {int(percentage.water.values)}%, unclassified = {int(unclass_percentage)}%")
           
    pixel_area_in_metres2 = 25 * 25
    m2_to_km2 = (1 / 1_000_000)
    percent_to_fraction = (1 / 100)
    
    area = (fc * (pixel_area_in_metres2 * m2_to_km2 * percent_to_fraction)).sum(dim=['x','y'])
    area = area.rename({'BS': 'BS_area', 
                        'PV': 'PV_area', 
                        'NPV': 'NPV_area', 
                        'water': 'water_area',
                        'mask_int':'mask_area',})
                        #'unclassified':'unclassified_area'})

    for da in area.data_vars.values():
        da.attrs['units'] = 'km^2'
    fc = percentage.merge(area)
    fc['average_data_count'] = has_data
    return fc

In [ ]:
def month_splitter(start_year, end_year_inclusive):
    """ 
    Split specified years into months 
    """
    yield from (str(p) for p in pd.period_range(start=pd.Period(start_year).start_time, 
                               end=pd.Period(end_year_inclusive).end_time, 
                               freq='M'))

In [ ]:
def output_csv(input_ds, sa2_id, sa2_name, sa2_size, albers_id, monthly_or_annual='frequency'):
    """
    Save tabulated data into a csv
    """
    input_ds = input_ds.to_dataframe()
    input_ds.insert(0,'SA2_id', sa2_id)
    input_ds.insert(1,'SA2_name', sa2_name)
    input_ds.insert(2,'albers_id', albers_id)
    input_ds.insert(3,'SA2_size', sa2_size)
    input_ds.to_csv(f"{output_path}/{name_of_output_file}_{monthly_or_annual}.csv",mode='a',header=False)  

## 7. Set up the query
For each year and polygon query the product, apply the geometry mask and compute the fractional cover stats

Using `client.compute()` lets us use the monthly results in calculating the annual results at the same time.

In [ ]:
#Obtain spatial information from shapefile
shape_file = os.path.expanduser(f'{shapefile_path}')
shapes = list(get_shapes(shape_file))

# #Specify a particular SA2 boundary, if required
#shapes = [(g,p) for g, p in shapes if str(p['SA2_MAIN16']) == '801111141']
# shapes = [(g,p) for g, p in shapes if str(p['AREASQKM16']) < '25_000']


print(f"Cell finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

If we have enough resources, we can start the query and calculation of the next year's data while the previous is still being calculated. `by_slice=False` will be faster, but use more memory.

For larger areas `by_slice` will need to be `True`, so that the compute cluster does not become overwhelmed.  

If you get the error:
> `distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting`

then you will need to set `by_slice=True`

In [ ]:
by_slice=True

In [ ]:
monthly_values = None
def process_area(geometry, sa2_id, sa2_name, sa2_size, albers_id, time_range):
    monthly_values = []
    annual_values = []
    mask = None
    
    # Virtual Products have a bug handling geometry objects, use this instead of `geopolygon=geometry` for now
    search_terms = {
        'x': (geometry.envelope.left, geometry.envelope.right),
        'y': (geometry.envelope.top, geometry.envelope.bottom),
        'crs': str(geometry.crs),
    }  

    for sub_time_range in month_splitter(time_range[0], time_range[-1]):
        print(sub_time_range)
        try:
            data = fc_and_water.load(dc, dask_chunks=chunk_size, 
                                     time=sub_time_range, 
                                     **search_terms)
        except ValueError:
            print(f'    No data for {sub_time_range} , skipping...')
            continue

        if mask is None:
            mask = geometry_mask([geometry], data.geobox, invert=True, chunks=data.chunks)
            mask_int = mask * 1
            mask_int = mask_int.sum() * 100

        data = data.where(mask)
        monthly_data = fc_and_water_summary(data, mask_int)
        monthly_data = client.compute(monthly_data, sync=by_slice)
        monthly_values.append(monthly_data)
        
    if not by_slice:
        print("  all years queried, hard load data")
        monthly_values = client.gather(monthly_values)
        
    monthly_values = xr.concat(monthly_values, dim='time')
    
#     monthly_qa = xr.Dataset(coords={'time': monthly_values.time})
#     monthly_qa['has_values'] = (('time',), [1] * monthly_values.time.size)
#     monthly_qa['has_10'] = monthly_values['unclassified'] <= 0
    
#     monthly_values = monthly_values.where(monthly_values['unclassified'] < 0).dropna(dim='time')
    
    annual_values = monthly_values.resample(time='1YS').mean(dim='time', skipna=True)
    
    # Replace the mean of the monthly obs count with the annual total obs count
    annual_avg_data_count = monthly_values['average_data_count'].resample(time='1YS').sum(dim='time', skipna=True)
    annual_values['average_data_count'] = annual_avg_data_count
    
#     annual_qa = monthly_qa.resample(time='1YS').sum(dim='time', skipna=True)
#     annual_qa = 12 - annual_qa
#     print(annual_qa)
#     annual_values = annual_values.merge(annual_qa)
    
    print(f"Calculation complete for annual values")
    
#     plot_stacked(monthly_values, sa2_id, plot_title='monthly_plot_wofs',show=False)
#     plot_stacked(annual_values, sa2_id, plot_title='annual_plot_wofs', show=False)
    
    print("All data loaded, save to csv")
    output_csv(monthly_values, sa2_id, sa2_name, sa2_size,albers_id, monthly_or_annual='monthly')
#     output_csv(annual_values, sa2_id, sa2_name, sa2_size,albers_id, monthly_or_annual='annual')
       
    print(f"SA2 {sa2_id} done")

## 8. Process query

In [ ]:
#Create empty CSV with specified headings
header = ['DATE','SA2_MAIN16', 'SA2_NAME16', 'ALBERS3577_ID', 'AREASQKM16',
          'PV_PERCENTAGE','NPV_PERCENTAGE','BS_PERCENTAGE','WOFL_PERCENTAGE',
          'PV_AREA_SQKM_ALBERS3577','NPV_AREA_SQKM_ALBERS3577',
          'BS_AREA_SQKM_ALBERS3577','WOFL_AREA_SQKM_ALBERS3577','FC_AREA_SQKM_ALBERS3577', 
          'AVERAGE_OBSERVATION_COUNT']
    
with open(f"{output_path}/{name_of_output_file}_monthly.csv","w") as outcsv: #create csv to save output and add header text
    writer = csv.writer(outcsv)
    writer.writerow(header)

In [ ]:
#Run tabulation script
for geometry, properties in shapes:
    sa2_id = str(properties['SA2_MAIN16'])
    sa2_name = str(properties['SA2_NAME16'])
    sa2_size = str(properties['AREASQKM16'])
    albers_id = str(properties['label'])
    print(f"SA2 ID: {sa2_id}, Albers tile: {albers_id}, SA2 size: {sa2_size}km^2, time: {time_range}")
    
    try:
        process_area(geometry, sa2_id, sa2_name, sa2_size, albers_id, time_range)
    except Exception as e:
          print(f"Could not process {sa2_id}: {e}")
          client.restart()

## Combine results for tiled SA2

In [8]:
df_in

,DATE,SA2_MAIN16,SA2_NAME16,ALBERS3577_ID,AREASQKM16,PV_PERCENTAGE,NPV_PERCENTAGE,BS_PERCENTAGE,WOFL_PERCENTAGE,PV_AREA_SQKM_ALBERS3577,NPV_AREA_SQKM_ALBERS3577,BS_AREA_SQKM_ALBERS3577,WOFL_AREA_SQKM_ALBERS3577,FC_AREA_SQKM_ALBERS3577,AVERAGE_OBSERVATION_COUNT
0,2005-01-01,113031266,Temora,"13,-39",3434.262,6.627565,51.645237,40.294571,0.001718,86.426540,673.47770,525.45966,0.022402,1304.046875,3.068191
1,2005-02-01,113031266,Temora,"13,-39",3434.262,9.008576,53.976929,35.619011,0.010097,117.475980,703.88440,464.48883,0.131667,1304.046875,3.094467
2,2005-03-01,113031266,Temora,"13,-39",3434.262,7.956395,46.712815,43.878465,0.001456,103.755080,609.15643,572.19570,0.018987,1304.046875,3.189311
3,2005-04-01,113031266,Temora,"13,-39",3434.262,7.206764,48.865057,42.547880,0.002651,93.979530,637.22350,554.84454,0.034568,1304.046875,3.648262
4,2005-05-01,113031266,Temora,"13,-39",3434.262,5.994199,43.307711,49.273543,0.000347,78.167145,564.75320,642.55040,0.004531,1304.046875,4.010324
5,2005-06-01,113031266,Temora,"13,-39",3434.262,9.703691,49.698819,37.947382,0.000024,126.540634,648.09644,494.85180,0.000313,1304.046875,1.825939
6,2005-07-01,113031266,Temora,"13,-39",3434.262,38.319006,43.684749,16.829472,0.000240,499.698100,569.66895,219.46417,0.003125,1304.046875,1.914579
7,2005-08-01,113031266,Temora,"13,-39",3434.262,50.548850,33.737031,14.377300,0.018115,659.180600,439.94675,187.48668,0.236229,1304.046875,2.367158
8,2005-09-01,113031266,Temora,"13,-39",3434.262,65.976539,24.643526,7.928755,0.048143,860.365300,321.36334,103.39473,0.627812,1304.046875,2.672486
9,2005-10-01,113031266,Temora,"13,-39",3434.262,62.158153,24.758560,11.410262,0.019994,810.571350,322.86328,148.79507,0.260729,1304.046875,2.362804


In [11]:
# df = df_in.groupby(['SA2_MAIN16','DATE','SA2_NAME16','AREASQKM16']).sum()/2
# df = df.reset_index(level=['DATE','AREASQKM16','SA2_NAME16'])
# df['DATE'] = pd.to_datetime(df['DATE'])


test = df.groupby([df.SA2_NAME16, df.DATE.dt.year]).sum()['AVERAGE_OBSERVATION_COUNT']
# annual_df = df.groupby([df.SA2_NAME16, df.DATE.dt.year]).mean()
test

SA2_NAME16  DATE
Temora      2005    40.631115
            2006    42.273824
Name: AVERAGE_OBSERVATION_COUNT, dtype: float64

In [ ]:
test = df.groupby([df.SA2_NAME16, df.DATE.dt.year]).sum()['AVERAGE_OBSERVATION_COUNT']
annual_df = df.groupby([df.SA2_NAME16, df.DATE.dt.year]).mean()
test

In [31]:
#read area results of previous workflow combine tiles for each SA2

df_in = pd.read_csv(f"{output_path}/{name_of_output_file}_monthly.csv", header=0)
df_in['DATE'] = pd.to_datetime(df_in['DATE'])
df = df_in[['DATE','SA2_MAIN16', 'SA2_NAME16','ALBERS3577_ID', 'AREASQKM16',
            'PV_AREA_SQKM_ALBERS3577','NPV_AREA_SQKM_ALBERS3577',
           'BS_AREA_SQKM_ALBERS3577','WOFL_AREA_SQKM_ALBERS3577',
            'FC_AREA_SQKM_ALBERS3577','AVERAGE_OBSERVATION_COUNT']]


df = df.groupby(['SA2_MAIN16','DATE','SA2_NAME16','AREASQKM16']).sum()


df['PV_PERCENTAGE']= df.PV_AREA_SQKM_ALBERS3577/df.FC_AREA_SQKM_ALBERS3577 *100
df['NPV_PERCENTAGE']= df.NPV_AREA_SQKM_ALBERS3577/df.FC_AREA_SQKM_ALBERS3577 *100
df['BS_PERCENTAGE']= df.BS_AREA_SQKM_ALBERS3577/df.FC_AREA_SQKM_ALBERS3577 *100
df['WOFL_PERCENTAGE']= df.WOFL_AREA_SQKM_ALBERS3577/df.FC_AREA_SQKM_ALBERS3577 *100
df['UNCLASSIFIED_PERCENTAGE']= 100 - (df.PV_PERCENTAGE + df.NPV_PERCENTAGE + df.BS_PERCENTAGE + df.WOFL_PERCENTAGE)
unclass_area = df['UNCLASSIFIED_PERCENTAGE'] * (df['FC_AREA_SQKM_ALBERS3577']/100)
df.insert(7,'UNCLASSIFIED_AREA_SQKM_ALBERS3577',unclass_area)
df = df.reset_index(level=['DATE','AREASQKM16','SA2_NAME16'])

#calculate the "no data" quality metric
no_data_qm = df.groupby([df.SA2_NAME16, df.DATE.dt.year]).count()
no_data_qm = no_data_qm.iloc[:,:1]
no_data_qm.rename(columns={'DATE':'MONTHS_WITH_NO_DATA'},inplace=True)
no_data_qm = 12 - no_data_qm

#remove all months that contain >10% unclassified
df = df.where(df.UNCLASSIFIED_PERCENTAGE < 10).dropna()

#take a mean of all months within each year
annual_df = df.groupby([df.SA2_NAME16, df.DATE.dt.year]).mean()

#calculate "low data" quality metric
low_data_qm = df.groupby([df.SA2_NAME16, df.DATE.dt.year]).count()
low_data_qm = low_data_qm.iloc[:,:1]
low_data_qm.rename(columns={'DATE':'MONTHS_WITH_LOW_DATA'},inplace=True)
low_data_qm = 12 - low_data_qm

#Concetenate fc, and quality metrics together
annual_df_with_qm = pd.concat([annual_df,low_data_qm, no_data_qm],axis=1,join='inner')

#calculate "average observation quality metric"
ave_obs_qm = df_in.groupby([df_in.SA2_NAME16, df_in.DATE.dt.year]).sum()['AVERAGE_OBSERVATION_COUNT'] 

# test = df_in.groupby([df_in.ALBERS3577_ID, df.SA2_NAME16, df_in.DATE.dt.year]).count()

annual_df_with_qm.pop('AVERAGE_OBSERVATION_COUNT')
annual_df_with_qm ['AVERAGE_OBSERVATION_COUNT'] = ave_obs_qm

annual_df_with_qm.to_csv(f'{output_path}/{name_of_output_file}_annual.csv')

In [59]:
df_in

,DATE,SA2_MAIN16,SA2_NAME16,ALBERS3577_ID,AREASQKM16,PV_PERCENTAGE,NPV_PERCENTAGE,BS_PERCENTAGE,WOFL_PERCENTAGE,PV_AREA_SQKM_ALBERS3577,NPV_AREA_SQKM_ALBERS3577,BS_AREA_SQKM_ALBERS3577,WOFL_AREA_SQKM_ALBERS3577,FC_AREA_SQKM_ALBERS3577,AVERAGE_OBSERVATION_COUNT
0,2005-01-01,113031266,Temora,"13,-39",3434.262,6.627565,51.645237,40.294571,0.001718,86.426540,673.47770,525.45966,0.022402,1304.046875,3.068191
1,2005-02-01,113031266,Temora,"13,-39",3434.262,9.008576,53.976929,35.619011,0.010097,117.475980,703.88440,464.48883,0.131667,1304.046875,3.094467
2,2005-03-01,113031266,Temora,"13,-39",3434.262,7.956395,46.712815,43.878465,0.001456,103.755080,609.15643,572.19570,0.018987,1304.046875,3.189311
3,2005-04-01,113031266,Temora,"13,-39",3434.262,7.206764,48.865057,42.547880,0.002651,93.979530,637.22350,554.84454,0.034568,1304.046875,3.648262
4,2005-05-01,113031266,Temora,"13,-39",3434.262,5.994199,43.307711,49.273543,0.000347,78.167145,564.75320,642.55040,0.004531,1304.046875,4.010324
5,2005-06-01,113031266,Temora,"13,-39",3434.262,9.703691,49.698819,37.947382,0.000024,126.540634,648.09644,494.85180,0.000313,1304.046875,1.825939
6,2005-07-01,113031266,Temora,"13,-39",3434.262,38.319006,43.684749,16.829472,0.000240,499.698100,569.66895,219.46417,0.003125,1304.046875,1.914579
7,2005-08-01,113031266,Temora,"13,-39",3434.262,50.548850,33.737031,14.377300,0.018115,659.180600,439.94675,187.48668,0.236229,1304.046875,2.367158
8,2005-09-01,113031266,Temora,"13,-39",3434.262,65.976539,24.643526,7.928755,0.048143,860.365300,321.36334,103.39473,0.627812,1304.046875,2.672486
9,2005-10-01,113031266,Temora,"13,-39",3434.262,62.158153,24.758560,11.410262,0.019994,810.571350,322.86328,148.79507,0.260729,1304.046875,2.362804


In [63]:
df_in.ALBERS3577_ID.groupby([df_in.ALBERS3577_ID, df_in.SA2_MAIN16]).count()

ALBERS3577_ID  SA2_MAIN16
13,-39         113031266     24
14,-39         113031266     24
Name: ALBERS3577_ID, dtype: int64

In [21]:
annual_df_with_qm

AREASQKM16  PV_AREA_SQKM_ALBERS3577  \
SA2_NAME16 DATE                                        
Temora     2005    3434.262               952.644076   
           2006    3434.262               689.225855   

                 NPV_AREA_SQKM_ALBERS3577  BS_AREA_SQKM_ALBERS3577  \
SA2_NAME16 DATE                                                      
Temora     2005               1559.529541               833.106477   
           2006               1823.439323               846.881459   

                 WOFL_AREA_SQKM_ALBERS3577  FC_AREA_SQKM_ALBERS3577  \
SA2_NAME16 DATE                                                       
Temora     2005                   1.018344               3434.26125   
           2006                   0.468736               3434.26125   

                 PV_PERCENTAGE  UNCLASSIFIED_AREA_SQKM_ALBERS3577  \
SA2_NAME16 DATE                                                     
Temora     2005      27.739418                          87.962812   
           2006      20.069115                          74.245877   

                 NPV_PERCENTAGE  BS_PERCENTAGE  WOFL_PERCENTAGE  \
SA2_NAME16 DATE                                                   
Temora     2005       45.410917      24.258681         0.029652   
           2006       53.095533      24.659786         0.013649   

                 UNCLASSIFIED_PERCENTAGE  MONTHS_WITH_LOW_DATA  \
SA2_NAME16 DATE                                                  
Temora     2005                 2.561331                     0   
           2006                 2.161917                     1   

                 MONTHS_WITH_NO_DATA  AVERAGE_OBSERVATION_COUNT  
SA2_NAME16 DATE                                                  
Temora     2005                    0                  81.262229  
           2006                    0                  84.547648

In [ ]:
annual_df_with_qm

In [ ]:
# monthly_values = None
# def process_area(geometry, sa2_id, sa2_name, sa2_size, albers_id, time_range):
#     monthly_values = []
#     annual_values = []
#     mask = None
    
#     # Virtual Products have a bug handling geometry objects, use this instead of `geopolygon=geometry` for now
#     search_terms = {
#         'x': (geometry.envelope.left, geometry.envelope.right),
#         'y': (geometry.envelope.top, geometry.envelope.bottom),
#         'crs': str(geometry.crs),
#     }  

#     for sub_time_range in month_splitter(time_range[0], time_range[-1]):
#         print(sub_time_range)
#         try:
#             data = fc_and_water.load(dc, dask_chunks=chunk_size, 
#                                      time=sub_time_range, 
#                                      **search_terms)
#         except ValueError:
#             print(f'    No data for {sub_time_range} , skipping...')
#             continue

#         if mask is None:
#             mask = geometry_mask([geometry], data.geobox, invert=True, chunks=data.chunks)
#             mask_int = mask * 1
#             mask_int = mask_int.sum() * 100

#         data = data.where(mask)
#         monthly_data = fc_and_water_summary(data, mask_int)
#         monthly_data = client.compute(monthly_data, sync=by_slice)
#         monthly_values.append(monthly_data)
        
#     if not by_slice:
#         print("  all years queried, hard load data")
#         monthly_values = client.gather(monthly_values)
        
#     monthly_values = xr.concat(monthly_values, dim='time')
    
# #     monthly_qa = xr.Dataset(coords={'time': monthly_values.time})
# #     monthly_qa['has_values'] = (('time',), [1] * monthly_values.time.size)
# #     monthly_qa['has_10'] = monthly_values['unclassified'] <= 0
    
# #     monthly_values = monthly_values.where(monthly_values['unclassified'] < 0).dropna(dim='time')
    
#     annual_values = monthly_values.resample(time='1YS').mean(dim='time', skipna=True)
    
#     # Replace the mean of the monthly obs count with the annual total obs count
#     annual_avg_data_count = monthly_values['average_data_count'].resample(time='1YS').sum(dim='time', skipna=True)
#     annual_values['average_data_count'] = annual_avg_data_count
    
# #     annual_qa = monthly_qa.resample(time='1YS').sum(dim='time', skipna=True)
# #     annual_qa = 12 - annual_qa
# #     print(annual_qa)
# #     annual_values = annual_values.merge(annual_qa)
    
#     print(f"Calculation complete for annual values")
    
# #     plot_stacked(monthly_values, sa2_id, plot_title='monthly_plot_wofs',show=False)
# #     plot_stacked(annual_values, sa2_id, plot_title='annual_plot_wofs', show=False)
    
#     print("All data loaded, save to csv")
#     output_csv(monthly_values, sa2_id, sa2_name, sa2_size,albers_id, monthly_or_annual='monthly')
# #     output_csv(annual_values, sa2_id, sa2_name, sa2_size,albers_id, monthly_or_annual='annual')
       
#     print(f"SA2 {sa2_id} done")

In [ ]:
# #Create empty CSV with specified headings
# header = ['DATE','SA2_MAIN16', 'SA2_NAME16', 'ALBERS3577_ID', 'AREASQKM16',
#           'PV_PERCENTAGE','NPV_PERCENTAGE','BS_PERCENTAGE','WOFL_PERCENTAGE',
#           'PV_AREA_SQKM_ALBERS3577','NPV_AREA_SQKM_ALBERS3577',
#           'BS_AREA_SQKM_ALBERS3577','WOFL_AREA_SQKM_ALBERS3577','TILE_AREA_SQKM_ALBERS3577', 
#           'AVERAGE_OBSERVATION_COUNT']

# # additional_annual_header = ['MONTHS_WITH_NO_DATA', 'MONTHS_WITH_LOW_DATA']

# # with open(f"{output_path}/{name_of_output_file}_annual.csv","w") as outcsv: #create csv to save output and add header text
# #     writer = csv.writer(outcsv)
# #     writer.writerow(header + additional_annual_header)
    
# with open(f"{output_path}/{name_of_output_file}_monthly.csv","w") as outcsv: #create csv to save output and add header text
#     writer = csv.writer(outcsv)
#     writer.writerow(header)

In [ ]:
# #read area results of previous workflow combine tiles for each SA2
# # Note, quality metrics aren't included at this point in time

# df_in = pd.read_csv(f"{output_path}/{name_of_output_file}_annual.csv", header=0)
# df_quality = df_in[['DATE','SA2_MAIN16', 'SA2_NAME16','ALBERS3577_ID', 'AREASQKM16','AVERAGE_OBSERVATION_COUNT',
#                   'MONTHS_WITH_NO_DATA','MONTHS_WITH_LOW_DATA']]
# # df_quality = df_quality.groupby(['SA2_MAIN16','DATE','SA2_NAME16','AREASQKM16']).sum()

# df = df_in[['DATE','SA2_MAIN16', 'SA2_NAME16','ALBERS3577_ID', 'AREASQKM16','PV_AREA_SQKM_ALBERS3577','NPV_AREA_SQKM_ALBERS3577',
#           'BS_AREA_SQKM_ALBERS3577','WOFL_AREA_SQKM_ALBERS3577','UNCLASSIFIED_AREA_SQKM_ALBERS3577']]
# df = df.groupby(['SA2_MAIN16','DATE','SA2_NAME16','AREASQKM16']).sum()
# # df = df.reset_index()

In [ ]:
# def fc_and_water_summary(data, mask_int):
#     """
#     Calculate the percentage and the area of each FC component, 
#     water, and unclassified data within each SA2 boundary. 
#     """
#     # Where there is water, set the FC bands to 0%
#     valid_data = numpy.isfinite(data['BS'])
#     fc = data[['PV', 'NPV', 'BS']].where(data.water==0, other=0)
#     fc['water'] = data.water.where(valid_data) * numpy.float32(100)
#     fc = fc.apply(lambda data_array: data_array.clip(0, 100).where(valid_data))

#     has_data = valid_data.groupby(valid_data.time.astype('datetime64[M]'), squeeze=False).sum(dim='time', skipna=False)
#     has_data = has_data.sum(dim=['x','y'], skipna=True) / (mask_int / 100)
    
#     # Flatten to a monthly mean
#     fc = fc.groupby(fc.time.astype('datetime64[M]'), squeeze=False).mean(dim='time', skipna=True)
#     # Calculate percentage of cover based on area of mask
#     percentage = fc.sum(dim=['x','y']) * (100 / mask_int)
#     for da in percentage.data_vars.values():
#         da.attrs['units'] = '%'

#     fc_sum = fc.sum(dim=['x','y'])
#     fc_sum = fc_sum.PV + fc_sum.NPV + fc_sum.BS + fc_sum.water
    
#     unclass_size = mask_int - fc_sum
#     unclass_percentage = unclass_size * (100 / mask_int)
#     fc['unclassified'] = unclass_percentage 
#     percentage['unclassified'] = unclass_percentage
    
# #     print(f"   PV = {int(percentage.PV.values)}%, NPV = {int(percentage.NPV.values)}%, BS = {int(percentage.BS.values)}%, water = {int(percentage.water.values)}%, unclassified = {int(unclass_percentage)}%")
           
#     pixel_area_in_metres2 = 25 * 25
#     m2_to_km2 = (1 / 1_000_000)
#     percent_to_fraction = (1 / 100)
    
#     area = (fc * (pixel_area_in_metres2 * m2_to_km2 * percent_to_fraction)).sum(dim=['x','y'])
#     area = area.rename({'BS': 'BS_area', 
#                         'PV': 'PV_area', 
#                         'NPV': 'NPV_area', 
#                         'water': 'water_area',
#                         'unclassified':'unclassified_area'})
    
#     for da in area.data_vars.values():
#         da.attrs['units'] = 'km^2'
#     fc = percentage.merge(area)
#     fc['average_data_count'] = has_data
#     return fc

In [ ]:
# def plot_stacked(ds, sa2_id,plot_title='title', show=True):
#     """
#     Create and save a stacked plot to visualise FC components
#     """
#     if not show:
#         plt.ioff()
        
#     fig,ax = plt.subplots(figsize=(10,5))
#     ax.stackplot(ds.dropna(dim='time').time.data, 
#                  ds.dropna(dim='time').PV,
#                  ds.dropna(dim='time').NPV, 
#                  ds.dropna(dim='time').BS, 
#                  ds.dropna(dim='time').water, 
#                  ds.dropna(dim='time').unclassified, 
#                  colors = ['darkolivegreen','olive','tan','darkblue','red'], 
#                  labels=['PV','NPV','BS','Water','Unclassified',])
#     plt.legend(loc='upper center', ncol = 5)
#     plt.title(f'FC components: SA2 ID {sa2_id}', size=12)
#     plt.ylabel('Percentage (%)', size=12) #Set Y label
#     plt.xlabel('Date', size=12) #Set X label
    
#     plt.savefig(f'{output_path}/{sa2_id}_{plot_title}.png');
#     plt.close(fig)
    
#     # Turn interactive back on
#     if not show:
#         plt.show()
# print(f"Cell finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# df_quality

In [ ]:
# monthly_values = None
# def process_area(geometry, sa2_id, sa2_name, sa2_size, albers_id, time_range):
#     monthly_values = []
#     annual_values = []
#     mask = None
    
#     # Virtual Products have a bug handling geometry objects, use this instead of `geopolygon=geometry` for now
#     search_terms = {
#         'x': (geometry.envelope.left, geometry.envelope.right),
#         'y': (geometry.envelope.top, geometry.envelope.bottom),
#         'crs': str(geometry.crs),
#     }  

#     for sub_time_range in month_splitter(time_range[0], time_range[-1]):
#         print(sub_time_range)
#         try:
#             data = fc_and_water.load(dc, dask_chunks=chunk_size, 
#                                      time=sub_time_range, 
#                                      **search_terms)
#         except ValueError:
#             print(f'    No data for {sub_time_range} , skipping...')
#             continue

#         if mask is None:
#             mask = geometry_mask([geometry], data.geobox, invert=True, chunks=data.chunks)
#             mask_int = mask * 1
#             mask_int = mask_int.sum() * 100
#             mask_int.load()

#         data = data.where(mask)
#         monthly_data = fc_and_water_summary(data, mask_int)
#         monthly_data = client.compute(monthly_data, sync=by_slice)
#         monthly_values.append(monthly_data)
        
#     if not by_slice:
#         print("  all years queried, hard load data")
#         monthly_values = client.gather(monthly_values)
        
#     monthly_values = xr.concat(monthly_values, dim='time')
    
#     monthly_qa = xr.Dataset(coords={'time': monthly_values.time})
#     monthly_qa['has_values'] = (('time',), [1] * monthly_values.time.size)
#     monthly_qa['has_10'] = monthly_values['unclassified'] <= 0
    
#     monthly_values = monthly_values.where(monthly_values['unclassified'] < 0).dropna(dim='time')
#     print(monthly_values) 
#     print(monthly_qa)
    
#     annual_values = monthly_values.resample(time='1YS').mean(dim='time', skipna=True)
    
#     # Replace the mean of the monthly obs count with the annual total obs count
#     annual_avg_data_count = monthly_values['average_data_count'].resample(time='1YS').sum(dim='time', skipna=True)
#     annual_values['average_data_count'] = annual_avg_data_count
    
#     annual_qa = monthly_qa.resample(time='1YS').sum(dim='time', skipna=True)
#     annual_qa = 12 - annual_qa
#     print(annual_qa)
#     annual_values = annual_values.merge(annual_qa)
    
#     print(f"Calculation complete for annual values")
    
#     plot_stacked(monthly_values, sa2_id, plot_title='monthly_plot_wofs',show=False)
#     plot_stacked(annual_values, sa2_id, plot_title='annual_plot_wofs', show=False)
    
#     print("All data loaded, save to csv")
#     output_csv(monthly_values, sa2_id, sa2_name, sa2_size,albers_id, monthly_or_annual='monthly')
#     output_csv(annual_values, sa2_id, sa2_name, sa2_size,albers_id, monthly_or_annual='annual')
       
#     print(f"SA2 {sa2_id} done")

In [ ]:
# blah = df_in.groupby(['SA2_MAIN16','DATE']).count()
# blah = blah.reset_index()
# blah = blah.loc[blah['DATE']=='2000-01-01'].loc[:,['SA2_MAIN16','SA2_NAME16']]

In [ ]:
annual_df_with_qm

In [ ]:
# df.count()

In [ ]:
# df['PV_PERCENTAGE']= df.PV_AREA_SQKM_ALBERS3577/df.AREASQKM16 *100 #XXX check SA2 AREA size. reprocess???
# df['NPV_PERCENTAGE']= df.NPV_AREA_SQKM_ALBERS3577/df.AREASQKM16 *100
# df['BS_PERCENTAGE']= df.BS_AREA_SQKM_ALBERS3577/df.AREASQKM16 *100
# df['WOFL_PERCENTAGE']= df.WOFL_AREA_SQKM_ALBERS3577/df.AREASQKM16 *100
# df['UNCLASSIFIED_PERCENTAGE']= 100 - (df.PV_PERCENTAGE + df.NPV_PERCENTAGE + df.BS_PERCENTAGE + df.WOFL_PERCENTAGE)

# no_data_qm = df.groupby([df.SA2_NAME16, df.DATE.dt.year]).count()
# no_data_qm = no_data_qm.iloc[:,:1]
# no_data_qm.rename(columns={'DATE':'MONTHS_WITH_NO_DATA'},inplace=True)
# no_data_qm = 12 - no_data_qm


# df.count()

In [ ]:
# df = df.where(df.UNCLASSIFIED_PERCENTAGE < 10).dropna()

In [ ]:
# df.count()

In [ ]:
# low_data_qm = df.groupby([df.SA2_NAME16, df.DATE.dt.year]).count()
# low_data_qm = low_data_qm.iloc[:,:1]
# low_data_qm.rename(columns={'DATE':'MONTHS_WITH_LOW_DATA'},inplace=True)
# low_data_qm = 12 - low_data_qm

# annual_df_with_qm = pd.concat([annual_df,low_data_qm, no_data_qm],axis=1)

In [ ]:
# annual_df = df.groupby([df.SA2_NAME16, df.DATE.dt.year]).mean()
# annual_df

In [ ]:
# xr_monthly = df.to_xarray()
# xr_monthly

In [ ]:
# xr_monthly.AREASQKM16

In [ ]:
# xr_monthly.isel(DATE=0,SA2_MAIN16=0,SA2_NAME16=1)

In [ ]:
# xr.Dataset.from_dataframe(df)

In [ ]:
# xr.Dataset(df)

In [ ]:
# xr_monthly = df.to_xarray()
# xr_monthly['PV_PERCENTAGE']= xr_monthly.PV_AREA_SQKM_ALBERS3577/xr_monthly.AREASQKM16 *100 #XXX check SA2 AREA size. reprocess???
# xr_monthly['NPV_PERCENTAGE']= xr_monthly.NPV_AREA_SQKM_ALBERS3577/xr_monthly.AREASQKM16 *100
# xr_monthly['BS_PERCENTAGE']= xr_monthly.BS_AREA_SQKM_ALBERS3577/xr_monthly.AREASQKM16 *100
# xr_monthly['WOFL_PERCENTAGE']= xr_monthly.WOFL_AREA_SQKM_ALBERS3577/xr_monthly.AREASQKM16 *100

# xr_monthly['UNCLASSIFIED_PERCENTAGE']= 100 - (xr_monthly.PV_PERCENTAGE + xr_monthly.NPV_PERCENTAGE + xr_monthly.BS_PERCENTAGE + xr_monthly.WOFL_PERCENTAGE)

# #START here: quality metric looking at months
# # xr_monthly = xr_monthly.where(xr_monthly['UNCLASSIFIED_PERCENTAGE'] <10).dropna(dim='DATE') #usually <10
# #then do wuality metric here